<a href="https://colab.research.google.com/github/AhmedBenAbdessalam/MNIST_simple_model/blob/main/MNIST_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Imports

In [71]:
import numpy as np
import tensorflow as tf

import tensorflow_datasets as tfds

## Data

In [72]:
mnist_dataset, mnist_info = tfds.load(name="mnist", with_info=True, as_supervised=True)

In [73]:
mnist_info.features

FeaturesDict({
    'image': Image(shape=(28, 28, 1), dtype=tf.uint8),
    'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=10),
})

In [74]:
mnist_train , mnist_test = mnist_dataset['test'] ,mnist_dataset['test']
num_validation_samples = 0.1 * mnist_info.splits['train'].num_examples
num_validation_samples = tf.cast(num_validation_samples , tf.int64)


In [75]:
num_test_samples = mnist_info.splits['test'].num_examples
num_test_samples = tf.cast(num_test_samples , tf.int64)

In [76]:
def scale(image, label):
  image = tf.cast(image, tf.float32)
  image /= 255.
  return image, label

In [77]:
scaled_train_and_validation_data = mnist_train.map(scale)

scaled_test_data = mnist_train.map(scale)

BUFFER_SIZE = 10000

shuffed_train_and_validation_data = scaled_train_and_validation_data.shuffle(BUFFER_SIZE)

validation_data = shuffed_train_and_validation_data.take(num_validation_samples)
train_data = shuffed_train_and_validation_data.skip(num_validation_samples)

In [78]:
BATCH_SIZE = 100
train_data = train_data.batch(BATCH_SIZE)
validation_data = validation_data.batch(num_validation_samples)
test_data = scaled_test_data.batch(num_test_samples)

validation_inputs , validation_targets = next(iter(validation_data))

##Model

In [110]:
input_size = 784
output_size = 10
hidden_layer_size = 200

model = tf.keras.Sequential([
                             tf.keras.layers.Flatten(input_shape=(28,28,1)),
                             tf.keras.layers.Dense(200,activation='relu'),
                             tf.keras.layers.Dense(150,activation='relu'),
                             tf.keras.layers.Dense(output_size,activation='softmax')
                            ])

In [111]:
adam = tf.keras.optimizers.Adam(learning_rate=0.1)

In [112]:
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy' , metrics=['accuracy'] )

###Training

In [113]:
NUM_EPOCHS = 20

model.fit(train_data,epochs=NUM_EPOCHS,validation_data=(validation_inputs,validation_targets),verbose=2)

Epoch 1/20
40/40 - 1s - loss: 0.9873 - accuracy: 0.7218 - val_loss: 0.3863 - val_accuracy: 0.8880 - 1s/epoch - 30ms/step
Epoch 2/20
40/40 - 1s - loss: 0.3441 - accuracy: 0.8945 - val_loss: 0.2789 - val_accuracy: 0.9212 - 670ms/epoch - 17ms/step
Epoch 3/20
40/40 - 1s - loss: 0.2370 - accuracy: 0.9300 - val_loss: 0.2334 - val_accuracy: 0.9322 - 786ms/epoch - 20ms/step
Epoch 4/20
40/40 - 1s - loss: 0.2137 - accuracy: 0.9375 - val_loss: 0.2058 - val_accuracy: 0.9352 - 777ms/epoch - 19ms/step
Epoch 5/20
40/40 - 1s - loss: 0.1805 - accuracy: 0.9495 - val_loss: 0.1595 - val_accuracy: 0.9562 - 807ms/epoch - 20ms/step
Epoch 6/20
40/40 - 1s - loss: 0.1373 - accuracy: 0.9600 - val_loss: 0.1320 - val_accuracy: 0.9630 - 774ms/epoch - 19ms/step
Epoch 7/20
40/40 - 1s - loss: 0.1344 - accuracy: 0.9620 - val_loss: 0.1339 - val_accuracy: 0.9595 - 826ms/epoch - 21ms/step
Epoch 8/20
40/40 - 1s - loss: 0.1179 - accuracy: 0.9695 - val_loss: 0.0990 - val_accuracy: 0.9710 - 668ms/epoch - 17ms/step
Epoch 9/20


## Test the model

In [115]:
test_loss, test_accuracy = model.evaluate(test_data)

1/1 [==============================] - 0s 381ms/step - loss: 0.0257 - accuracy: 0.9955


In [117]:
print('Test loss: {0:.2f}. Test accuracy: {1:.2f}%'.format(test_loss,test_accuracy*100.))

Test loss: 0.03. Test accuracy: 99.55%
